In [1]:
# import Libraries
import pandas as pd
from sqlalchemy import create_engine

## Import Sales Data

In [2]:
# Create a dataframe 
df_sales = pd.read_csv('sales_data_sample.csv', encoding = 'latin1')
df_sales.head(5)


,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,2,2003,...,897 Long Airport Avenue,NaN,NYC,NY,10022,USA,NaN,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,5,2003,...,59 rue de l'Abbaye,NaN,Reims,NaN,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,7,2003,...,27 rue du Colonel Pierre Avia,NaN,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,8,2003,...,78934 Hillside Dr.,NaN,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,...,7734 Strong St.,NaN,San Francisco,CA,NaN,USA,NaN,Brown,Julie,Medium


## Data Understanding 

In [3]:
len(df_sales['ORDERDATE'])

2823

### Extract sales date related data from Sales to populate stg_date

#### Date Data

In [5]:
df_date = df_sales [['ORDERDATE','QTR_ID', 'MONTH_ID','YEAR_ID']]
df_date.head(5)

,ORDERDATE,QTR_ID,MONTH_ID,YEAR_ID
0,2/24/2003 0:00,1,2,2003
1,5/7/2003 0:00,2,5,2003
2,7/1/2003 0:00,3,7,2003
3,8/25/2003 0:00,3,8,2003
4,10/10/2003 0:00,4,10,2003


In [6]:
# make a copy of df_DimDate
df_date = df_date.copy()
# convert orderdate to datetime
df_date['ORDERDATE'] = pd.to_datetime(df_date['ORDERDATE'])
# drop time part
df_date['ORDERDATE'] = df_date['ORDERDATE'].dt.date

In [7]:
df_date['ORDERDATE'].head()

0    2003-02-24
1    2003-05-07
2    2003-07-01
3    2003-08-25
4    2003-10-10
Name: ORDERDATE, dtype: object

#### Populate stg_date table

##### Create a connection to SQL Srever

In [2]:
server = 'DB_SERVER'
database = 'DB_NAME'
#username = ''
#password = '
conn_str = f'mssql+pyodbc://@{server}/{database}?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server'
engine = create_engine(conn_str)
engine

Engine(mssql+pyodbc://@DB_SERVER/DB_NAME?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes)

##### Upload the Data to SQL Table

In [9]:
# column mapping 
column_mapping=  {'ORDERDATE':'order_date',
               'QTR_ID': 'qtr',
               'MONTH_ID': 'month',
               'YEAR_ID': 'year'}

In [10]:
df_date= df_date.rename(columns = column_mapping )
df_date

,order_date,qtr,month,year
0,2003-02-24,1,2,2003
1,2003-05-07,2,5,2003
2,2003-07-01,3,7,2003
3,2003-08-25,3,8,2003
4,2003-10-10,4,10,2003
...,...,...,...,...
2818,2004-12-02,4,12,2004
2819,2005-01-31,1,1,2005
2820,2005-03-01,1,3,2005
2821,2005-03-28,1,3,2005


In [12]:
no_records = df_date.to_sql('stg_date', con = engine, if_exists = 'append', index = False)
print(no_records)

203
